# Part 2 - Softmax Regression

The goal of this part of the project is to implement Softmax Regression in order to classify the MNIST digit dataset. Softmax Regression is essentially a two-layer neural network where the output layer applies the Softmax cost function, a multiclass generalization of the logistic cost function.

In logistic regression, we have a hypothesis function of the form

$$P[y=1] = \frac{1}{1 + e^{-wx}}$$

where $\mathbf{w}$ is our weight vector. Like the hyperbolic tangent function, the logistic function is also a sigmoid function with the characteristic 's'-like shape, though it has a range of $(0, 1)$ instead of $(-1, 1)$. Note that this is technically not a classifier since it returns probabilities instead of a predicted class, but it's easy to turn it into a classifier by simply choosing the class with the highest probability.

Since logistic regression is used for binary classification, it is easy to see that:

$$\begin{aligned}
P[y=1] &= \frac{1}{1 + e^{-wx}} \\
&= \frac{e^{wx}}{e^{wx} + e^{0 \cdot x}} \\
\end{aligned}$$

Similarily,

$$\begin{aligned}
P[y=0] &= 1 - P[y=1] \\
&= 1 - \frac{e^{wx}}{1 + e^{wx}} \\
&= \frac{e^{0 \cdot x}}{e^{wx} + e^{0 \cdot x}} \\
\end{aligned}$$

From this form it appears that we can assign the vector $w_1 = w$ as the weight vector for class $1$ and $w_0 = \vec{0}$ as the weight vector for class $0$. This motivates generalization to classification with more than 2 classes. By assigning a separate weight vector $w_i$ to each class, for each example $x$ we can predict the probability that it is class $i$, and again we can classify by choosing the most probable class.

$$P[y=i] = \frac{e^{w_i x}}{\sum\limits_j e^{w_j x}}$$

A more compact way of representing the values $w_i x$ is $Wx$ where each row $i$ of $W$ is $w_i$. We can also represent a dataset $\{x_i\}$ with a matrix $X$ where each column is a single example, resulting in $WX$.

## Part 2.1 - Questions [5%]

1. Show that the probabilities sum to 1.

**ANSWER**:

2. What are the dimensions of $W$, $X$, and $WX$?

**ANSWER**: